# Installs

In [ ]:
%pip install pycryptodome
%pip install cryptography

: 

# Hashing

## using pycryptodome

In [ ]:
from Crypto.Hash import SHA3_256, BLAKE2s, RIPEMD160, keccak

msg = "Hello"
h1 = SHA3_256.new()
h2 = BLAKE2s.new()
h3 = RIPEMD160.new()
h4 = keccak.new(digest_bits=512)

h1.update(msg.encode())
h2.update(msg.encode())
h3.update(msg.encode())
h4.update(msg.encode())

print(f"SHA3-256   of '{msg}': {h1.hexdigest()}")
print(f"BLAKE2s    of '{msg}': {h2.hexdigest()}")
print(f"RIPEMD160  of '{msg}': {h3.hexdigest()}")
print(f"Keccak-512 of '{msg}': {h4.hexdigest()}")

: 

## using Hashlib

In [ ]:
import hashlib

msg = "Hello"

# print(hashlib.algorithms_available)
h1 = hashlib.new('sha3_512')
h2 = hashlib.new('blake2s')
h3 = hashlib.new('ripemd160')

h1.update(msg.encode())
h2.update(msg.encode())
h3.update(msg.encode())

print(f"SHA3-256   of '{msg}': {h1.hexdigest()}")
print(f"BLAKE2s    of '{msg}': {h2.hexdigest()}")
print(f"RIPEMD160  of '{msg}': {h3.hexdigest()}")

: 

# Diffie-Hellman Key Exchange with sign

In [ ]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh, dsa
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import serialization


: 

In [ ]:
param = dh.generate_parameters(generator=2, key_size=2048)
pvt1 = param.generate_private_key()
pub1 = pvt1.public_key()
pvt2 = param.generate_private_key()
pub2 = pvt2.public_key()
print(f"Pvt-Pub key pair for encryption generated for both parties")

: 

In [ ]:
sign_pvt1 = dsa.generate_private_key(key_size=2048)
sign_pub1 = sign_pvt1.public_key()
sign_pvt2 = dsa.generate_private_key(key_size=2048)
sign_pub2 = sign_pvt2.public_key()
print(f"Signing key generated for both parties")

: 

In [ ]:
pub1_bytes = pub1.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo,
)
pub2_bytes = pub2.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo,
)
sign1 = sign_pvt1.sign(pub1_bytes, hashes.SHA3_256())
sign2 = sign_pvt2.sign(pub2_bytes, hashes.SHA3_256())
print(f"Signatures generated")

: 

In [ ]:
sign_pub1.verify(
    signature=sign1,
    data=pub1_bytes,
    algorithm=hashes.SHA3_256(),
)
sign_pub2.verify(
    signature=sign2,
    data=pub2_bytes,
    algorithm=hashes.SHA3_256(),
)
print("Signatures verified")

: 

In [ ]:
shared1 = pvt1.exchange(pub2)
shared2 = pvt2.exchange(pub1)
print(f" Shared secrets match: {shared1==shared2}")

: 

# Key Derivation

## PBKDF2

In [ ]:
from Crypto.Protocol.KDF import PBKDF2
from Crypto.Hash import SHA3_512
from Crypto.Random import get_random_bytes

password_ = "a secret"
salt_ = get_random_bytes(16)
key = PBKDF2(
    password=password_,
    salt=salt_,
    dkLen=64,
    count=1000000,
    hmac_hash_module=SHA3_512,
)
print(f"Password: {password_}")
print(f"Random salt: {salt_.hex()}")
print(f"Key: {key.hex()}")

: 

## Scrypt

In [ ]:
from Crypto.Protocol.KDF import scrypt
from Crypto.Random import get_random_bytes

password_ = "some secret"
salt_ = get_random_bytes(16)
key = scrypt(
    password=password_,
    salt=salt_,
    key_len=64,
    N=(1 << 10),
    r=8,
    p=1,
)
print(f"Password: {password_}")
print(f"Random salt: {salt_.hex()}")
print(f"Key: {key.hex()}")

: 

# Encryption and Decryption

## AES-256-GCM

In [20]:
from Crypto.Cipher import AES
import hashlib

msg = "Message for aes-256-gcm encryption"
pwd = "something"
hasher = hashlib.new('sha3_256')
hasher.update(pwd.encode())
pwd_hash = hasher.digest()

aes_cipher = AES.new(pwd_hash, AES.MODE_GCM)
ct, auth_tag = aes_cipher.encrypt_and_digest(msg.encode())
iv = aes_cipher.nonce

aes_decipher = AES.new(pwd_hash, AES.MODE_GCM, iv)
pt = aes_decipher.decrypt(ct)

print(f"Cipher text: {ct.hex()}")
print(f"Auth Tag: {auth_tag.hex()}")
print(f"Nonce: {iv.hex()}")
print(f"Password: {pwd}")

try:
    aes_decipher.verify(auth_tag)
    print(f"Tag verified!")
    print(f"Plain text: {pt.decode()}")
except:
    print(f"Msg tampered!")
    print(f"Plain text: {pt}")


Cipher text: d10e4c54171c5d83529780550a42b6f0a459ea6674c5ba3cd142e25cc0e8d46d0cae
Auth Tag: 8b9cf78879f8f0ebab2ec796bfa290bb
Nonce: 19b0f3ea10de3c9e15980ceae412c1d0
Password: something
Tag verified!
Plain text: Message for aes-256-gcm encryption


## AES-256-CBC

In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

msg = "Message for aes-256-cbc encryption"
pwd = "something"
hasher = hashlib.new("sha256")
hasher.update(pwd.encode())
pwd_hash = hasher.digest()

aes_cipher = AES.new(pwd_hash, AES.MODE_CBC)
ct = aes_cipher.encrypt(pad(msg.encode(), AES.block_size, style="pkcs7"))
iv = aes_cipher.iv

aes_decipher = AES.new(pwd_hash, AES.MODE_CBC, iv)
pt = unpad(aes_decipher.decrypt(ct), AES.block_size, "pkcs7")

print(f"Cipher text: {ct.hex()}")
print(f"IV: {iv.hex()}")
print(f"Password: {pwd}")
print(f"Pwd hash: {pwd_hash.hex()}")
print(f"Plain Text: {pt}")

: 

## AES-256-CTR

In [17]:
from Crypto.Cipher import AES

msg = "Message for aes-256-cbc encryption"
pwd = "something"
hasher = hashlib.new("sha3_256")
hasher.update(pwd.encode())
pwd_hash = hasher.digest()

aes_cipher = AES.new(pwd_hash, AES.MODE_CTR)
ct = aes_cipher.encrypt(msg.encode())
nonce = aes_cipher.nonce

aes_decipher = AES.new(pwd_hash, AES.MODE_CTR, nonce=nonce)
pt = aes_decipher.decrypt(ct)

print(f"Cipher text: {ct.hex()}")
print(f"IV: {nonce.hex()}")
print(f"Password: {pwd}")
print(f"Plain Text: {pt}")

NameError: name 'hashlib' is not defined

## DES

In [ ]:
# from Crypto.Cipher import DES3


# while True:
#     try:
#         key = DES3.adjust_key_parity(get_random_bytes(24))
#     except:
#         pass

# des_cipher = DES3.new(key, DES3.MODE_CBC)


: 

## ChaCha20

In [ ]:
from Crypto.Cipher import ChaCha20

msg = "Message for ChaCha20 encryption"
pwd = "something"
hasher = hashlib.new("sha3_256")
hasher.update(pwd.encode())
pwd_hash = hasher.digest()

chacha_cipher = ChaCha20.new(key=pwd_hash)
ct = chacha_cipher.encrypt(msg.encode())
nonce = chacha_cipher.nonce

chacha_decipher = ChaCha20.new(key=pwd_hash, nonce=chacha_cipher.nonce)
pt = chacha_decipher.decrypt(ct)

print(f"Cipher text: {ct.hex()}")
print(f"IV: {nonce.hex()}")
print(f"Password: {pwd}")
print(f"Plain Text: {pt}")

: 

## RSA with PKCS1 OEAP

In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

msg = "Message for ChaCha20 encryption"

key_pair = RSA.generate(1024)
pub_key = key_pair.public_key()
# print(pub_key.export_key())

n = key_pair.n
e = key_pair.e
d = key_pair.d

key_pair1 = RSA.construct((n, e, d))
key_pair2 = RSA.import_key(key_pair.export_key())
# print(f"Is key_pair == key pair_1: {key_pair==key_pair1}")
# print(f"Is key_pair == key pair_2: {key_pair==key_pair2}")

rsa_cipher = PKCS1_OAEP.new(key=pub_key)
ct = rsa_cipher.encrypt(msg.encode())

rsa_decipher = PKCS1_OAEP.new(key=key_pair)
pt = rsa_decipher.decrypt(ct)

print(f"Pub key: (n,e): ({pub_key.n},{pub_key.e})")
print(f"Pvt key: (n,d): ({key_pair.n},{key_pair.d})")
print(f"Cipher text: {ct.hex()}")
print(f"Plain text: {pt}")

: 

## ECC with AES-256-GCM (hybrid)

In [ ]:
from Crypto.PublicKey import ECC
from Crypto.PublicKey.ECC import EccPoint
from Crypto.Hash import SHA3_256
from Crypto.Cipher import AES

# key_pair = ECC.generate(curve="p521")
# pub_key = key_pair.public_key()
pvt1 = ECC.generate(curve="p521")
pvt2 = ECC.generate(curve="p521")
pub1 = pvt1.public_key()
pub2 = pvt2.public_key()

shared1 = pub2.pointQ * pvt1.d
shared2 = pub1.pointQ * pvt2.d
assert shared1 == shared2


def get_key(a: EccPoint):
    hasher = SHA3_256.new(a.x.to_bytes())
    hasher.update(a.y.to_bytes())
    return hasher.hexdigest()

msg = "I am ironman"
key = get_key(shared1)

aes_cipher = AES.new(bytes.fromhex(key), AES.MODE_GCM)
ct, auth_tag = aes_cipher.encrypt_and_digest(msg.encode())
nonce_ = aes_cipher.nonce

aes_decipher = AES.new(bytes.fromhex(key), AES.MODE_GCM, nonce=nonce_)
pt = aes_decipher.decrypt(ct)

try:
    aes_decipher.verify(auth_tag)
    print(f"Msg verified: {pt}")
except:
    print(f"Msg not verified: {pt}")


: 

# Signatures

## with RSA, PKCS1 v1.5

In [4]:
from Crypto.Signature import PKCS1_v1_5
from Crypto.Hash import SHA3_256
from Crypto.PublicKey import RSA

key = RSA.generate(1024)
msg = "I am inevitable"

pwd = b"something"
with open("rsa_key.pem", "wb") as f:
    data = key.export_key(
        passphrase=pwd,
        pkcs=8,
        protection="PBKDF2WithHMAC-SHA512AndAES256-CBC",
        prot_params={"iteration_count": 131072},
    )
    f.write(data)

with open("rsa_key.pem", "rb") as f:
    data = f.read()
    key_pair = RSA.import_key(data, pwd)

# assert key_pair == key

# Sender's side
pub_key = key_pair.public_key()
signer = PKCS1_v1_5.new(key_pair)
sign = signer.sign(SHA3_256.new(msg.encode()))

# Receiver's side
validator = PKCS1_v1_5.new(pub_key)
val = validator.verify(SHA3_256.new(msg.encode()), sign)
print(f"Signature validity: {val}")

Signature validity: True


## with ECC, DSS

In [2]:
from Crypto.PublicKey import ECC
from Crypto.Hash import SHA3_256
from Crypto.Signature import DSS

ecc_key = ECC.generate(curve="p256")

# pwd = "something"
# with open("ecc_key.pem", "wt") as f:
#     data = ecc_key.export_key(
#         format="PEM",
#         # passphrase=pwd.encode(),
#         # protection="PBKDF2WithHMAC-SHA512AndAES256-CBC",
#         # prot_params={"iteration_count": 131072},
#     )
#     f.write(data)
# with open("ecc_key.pem", "rt") as f:
#     data = f.read()
#     key_pair = ECC.import_key(data)

msg = "I am inevitable"
msg1 = "asdf"

# Sender's side
pub_key = key_pair.public_key()
signer = DSS.new(key=key_pair, mode="fips-186-3")
sign = signer.sign(SHA3_256.new(msg.encode()))

# Receiver's side
verifier = DSS.new(key=pub_key, mode='fips-186-3')
try:
    verifier.verify(SHA3_256.new(msg.encode()), sign)
    print("Signature is valid!")
except:
    print("Invalid Sign")

NameError: name 'key_pair' is not defined

In [3]:
msg = "test message"
nn = "jb"

pvt_key = ECC.generate(curve="p256")
pub_key = pvt_key.public_key()

with open(f"{nn}__ecc_pvt_key.pem", "wt") as f:
    f.write(pvt_key.export_key(format="PEM"))
with open(f"{nn}__ecc_pub_key.pem", "wt") as f:
    f.write(pub_key.export_key(format="PEM"))

with open(f"{nn}__ecc_pvt_key.pem", "rt") as f:
    key1 = ECC.import_key(f.read())
with open(f"{nn}__ecc_pub_key.pem", "rt") as f:
    key2 = ECC.import_key(f.read())

print(pvt_key == key1)
print(pub_key == key2)

signer = DSS.new(key=key1, mode="fips-186-3")
sign = signer.sign(SHA3_256.new(msg.encode())).hex()


True
True


In [4]:
# Receiver's side
verifier = DSS.new(key=key2, mode='fips-186-3')
try:
    verifier.verify(SHA3_256.new(msg.encode()), bytes.fromhex(sign))
    print("Signature is valid!")
except:
    print("Invalid Sign")

Signature is valid!
